### Задание

1. На данном датасете обучить модель https://huggingface.co/intfloat/multilingual-e5-small с помощью этого кода.
2. Переписать код под актуальную версию sentence-transformers.
3. Сравнить результаты

## Пример выполнения задания смотрите в мастер-классе по теме

In [ ]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 13.8 MB/s eta 0:00:00


In [ ]:
import torch

from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation, util
from torch.utils.data import DataLoader

from datasets import load_dataset
from sklearn.model_selection import train_test_split
import pickle
from random import shuffle

from statistics import mean
from torchmetrics.functional.retrieval import retrieval_reciprocal_rank, retrieval_average_precision, \
    retrieval_normalized_dcg, retrieval_recall

In [ ]:
device = 'mps' if torch.backends.mps.is_built() else 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


## Служебные функции для работы с данными

In [ ]:
# удаление повторяющихся документов
def shrink_repeated_samples(
    queries,
    docs,
    labels,
):
    docs_was = set()
    qs = []
    ds = []
    ls = []
    for i in range(len(queries)):
        q, d, l = queries[i], docs[i], labels[i]
        if d in docs_was:
            continue
        qs.append(q)
        ds.append(d)
        ls.append(l)
        docs_was.add(d)

    return qs, ds, ls

In [ ]:
# загрузка и перемешивание train
def train_shuffled_data(from_file=False):
    datafile = 'trainval.pkl'

    if from_file:
        with open(datafile, 'rb') as f:
            data = pickle.load(f)
        return data

    good, bad = load_train_samples()
    data = good + bad
    shuffle(data)

    with open(datafile, 'wb') as f:
        pickle.dump(data, f)

    return data

In [ ]:
# загрузка и перемешивание test
def test_shuffled_data(from_file=False):
    datafile = 'test.pkl'

    if from_file:
        with open(datafile, 'rb') as f:
            data = pickle.load(f)
        return data

    good, bad = load_test_samples()
    data = good + bad


    with open(datafile, 'wb') as f:
        pickle.dump(data, f)

    return data

In [ ]:
# формирование input example
def train_examples():
    data = train_shuffled_data(from_file=False)
    data = data[:25000]

    trainval = []
    for question, document, cos in data:
        trainval.append(
            InputExample(texts=[question, document], label=cos)
        )

    return trainval

In [ ]:
# загрузка тестовых примеров
def load_test_samples():
    _, test = _load_dataset()

    questions = test['question'].tolist()
    contexts = test['answer'].tolist()

    return _inflate_with_negative_samples(
        questions=questions,
        contexts=contexts,
    )

In [ ]:
# загрузка train
def load_train_samples():
    train, _ = _load_dataset()

    questions = train['question'].tolist()
    contexts = train['answer'].tolist()

    return _inflate_with_negative_samples(
        questions=questions,
        contexts=contexts,
    )

In [ ]:
# генерация негативных примеров (негативное семплирование)
def _inflate_with_negative_samples(
    questions,
    contexts,
    delta=30,
):
    good_q_c_cos = list(zip(questions, contexts, [1.0] * len(questions)))
    bad_q_c_cos = []
    n = len(good_q_c_cos)

    for i in range(n):
        cur_q, cur_c, _ = good_q_c_cos[i]
        next_q, next_c, _ = good_q_c_cos[(i + delta) % n]
        if next_c != cur_c:
            bad_q_c_cos.append((cur_q, next_c, 0.0))

    return good_q_c_cos, bad_q_c_cos

In [ ]:
# загрузка датасета
def _load_raw_dataset(from_file=False):
    raw_dataset = 'raw_dataset.pkl'
    if from_file:
        with open(raw_dataset, 'rb') as f:
            df = pickle.load(f)
    else:
        df = load_dataset("Den4ikAI/russian_instructions_2")
        df['train'] = df['train'].select(range(50000))
        with open(raw_dataset, 'wb') as f:
            pickle.dump(df, f)

    return df

In [ ]:
# разбиение на train, test
def _load_dataset():
    df = _load_raw_dataset()
    raw_data = df['train'].to_pandas()
    train, test = train_test_split(raw_data, test_size=0.2, random_state=42)

    return train, test

## Метрики "из коробки"

In [ ]:
from sentence_transformers import SentenceTransformer, models

def raw_bi_encoder():
    word_embedding_model = models.Transformer('intfloat/multilingual-e5-small', max_seq_length=256)
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension()) # дополнительно используем слой пулинга
    # для получения векторного представления текста целиком, а не токенов

    bi_encoder = SentenceTransformer(
        modules=[word_embedding_model, pooling_model],
        device=device,
    )

    return bi_encoder

In [ ]:
bi_encoder = raw_bi_encoder()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

In [ ]:
bi_encoder

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

Метрики близости без дообучения на test

In [ ]:
train = train_shuffled_data(from_file=False)

README.md: 0.00B [00:00, ?B/s]

dataset.jsonl:   0%|          | 0.00/233M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/237281 [00:00<?, ? examples/s]

In [ ]:
train_questions, train_contexts, train_coss = zip(*train)

In [ ]:
evaluator = evaluation.EmbeddingSimilarityEvaluator(train_questions, train_contexts, train_coss)

In [ ]:
bi_encoder.evaluate(evaluator)

{'pearson_cosine': 0.8811549276560331, 'spearman_cosine': 0.8568613438699423}

Метрики близости без дообучения на test

In [ ]:
bi_encoder = raw_bi_encoder()

In [ ]:
test = test_shuffled_data(from_file=False)

In [ ]:
shuffle(test)

In [ ]:
n_test_samples = 1000
test_questions, test_contexts, test_coss = zip(*test[:n_test_samples])

In [ ]:
evaluator = evaluation.EmbeddingSimilarityEvaluator(test_questions, test_contexts, test_coss)

In [ ]:
bi_encoder.evaluate(evaluator)

{'pearson_cosine': 0.8744010056847706, 'spearman_cosine': 0.8550462825683011}

Ожидаемо на необученной модели метрики для обучающих и тестовых данных примерно совпадают

# Дообучение

In [ ]:
train_examples = train_examples()

In [ ]:
print(train_examples[0].__dict__)

{'guid': '', 'texts': ['Какие продукты лучше всего употреблять для здорового питания сердца?', 'Лучшие продукты для здорового питания сердца включают фрукты, овощи, цельнозерновые продукты, нежирные белки и обезжиренные молочные продукты. Употребление в пищу большого количества растительных продуктов, богатых клетчаткой, таких как бобовые, орехи, семена и цельнозерновые продукты, может помочь снизить уровень холестерина. Кроме того, сокращение потребления насыщенных жиров и трансжиров важно для поддержания здорового уровня сахара в крови и поддержания здоровья сердечно-сосудистой системы.'], 'label': 1.0}


Позитивный семпл

In [ ]:
train_loader = DataLoader(
    train_examples,
    shuffle=True,
    batch_size=32,
    collate_fn=bi_encoder.smart_batching_collate # special batch + tokenizer
)

In [ ]:
train_loss_set = losses.CosineSimilarityLoss(bi_encoder)

In [ ]:
train_loss_set

CosineSimilarityLoss(
  (model): SentenceTransformer(
    (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
    (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  )
  (loss_fct): MSELoss()
  (cos_score_transformation): Identity()
)

Добавили лосс к модели

Формирование батча

In [ ]:
(query_batch, context_batch), labels = next(iter(train_loader))

In [ ]:
query_batch['input_ids'].shape, context_batch['input_ids'].shape, labels.shape

(torch.Size([32, 78]), torch.Size([32, 256]), torch.Size([32]))

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

Запуск дообучения

In [ ]:
bi_encoder.fit(
    train_objectives=[(train_loader, train_loss_set)],
    output_path='qa/results',
    epochs=3,
    evaluator=evaluator,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

## Загрузка чекпоинта

In [ ]:
finetuned_bi_encoder = SentenceTransformer('qa/results')

In [ ]:
finetuned_bi_encoder.evaluate(evaluator)

{'pearson_cosine': 0.9233526188449115, 'spearman_cosine': 0.8547684728308436}

# Получение предсказаний модели и меток


In [ ]:
test = test_shuffled_data(from_file=True)
n_test_samples = 1000
test_questions, test_contexts, labels = zip(*test[:n_test_samples])

print(f'docs count before shrinking: {len(test_contexts)}')

queries, docs, labels = shrink_repeated_samples(
    queries=test_questions,
    docs=test_contexts,
    labels=labels,
)

print(f'docs count without duplicates: {len(test_contexts)}')

docs count before shrinking: 1000
docs count without duplicates: 1000


Получение эмбеддингов (кодирование)

In [ ]:
context_embs = finetuned_bi_encoder.encode(
    test_contexts,
    convert_to_tensor=True,
    show_progress_bar=True,
)

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
question_embs = finetuned_bi_encoder.encode(
    test_questions,
    convert_to_tensor=True,
    show_progress_bar=True,
)

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

# Семантический поиск

In [ ]:
cos_scores = util.semantic_search(question_embs, context_embs, top_k=100)

In [ ]:
top_k = 5
q_idx = 0

In [ ]:
print(cos_scores[q_idx][:top_k])

[{'corpus_id': 660, 'score': 0.9122371673583984}, {'corpus_id': 0, 'score': 0.8851485252380371}, {'corpus_id': 985, 'score': 0.8700286746025085}, {'corpus_id': 997, 'score': 0.8678606748580933}, {'corpus_id': 513, 'score': 0.8397399187088013}]


In [ ]:
print('Question: ', test_questions[q_idx])
print()

for no, ir in enumerate(cos_scores[q_idx][:top_k]):
    corpus_id = ir["corpus_id"]
    print('corpus id: ', corpus_id)
    print(f'Document {no + 1}: Cosine Similarity is {ir["score"]:.3f}:\n\n{test_contexts[corpus_id]}')
    print()

Question:  У меня проблемы с весом. Есть ли что-то, что я могу сделать, чтобы потерять часть этого?

corpus id:  660
Document 1: Cosine Similarity is 0.912:

Вы можете попытаться сосредоточиться на здоровом питании и регулярно заниматься спортом. Это поможет вам оставаться активным и сжечь лишние калории. Кроме того, обязательно пейте много воды и высыпайтесь каждую ночь. Эти шаги могут помочь вам поддерживать более здоровый вес с течением времени.

corpus id:  0
Document 2: Cosine Similarity is 0.885:

Вы можете попробовать регулярно заниматься спортом. Выполнение физических упражнений, таких как бег, плавание или езда на велосипеде, поможет вам сжечь калории и ускорить метаболизм. Вы также можете сосредоточиться на здоровом питании, уменьшив количество обработанных продуктов и сладких закусок в своем рационе, а также добавив больше фруктов и овощей. Кроме того, употребление большого количества воды и достаточный сон важны для поддержания здорового веса.

corpus id:  985
Document 3: C

# Расчёт метрик: Recall@5, MRR, MAP, NDCG@10

In [ ]:
def build_target_mask_for_i(i, n):
    target_mask = [False] * n
    target_mask[i] = True
    return target_mask

In [ ]:
def build_pred_mask_for_i(
    cos_scores,
    i,
    n,
):
    pred_mask = [0.0] * n
    qi_scores = cos_scores[i]

    for docid_score in qi_scores:
        doc_id = docid_score['corpus_id']
        score = docid_score['score']
        pred_mask[int(doc_id)] = score

    return pred_mask

### До дообучения

In [ ]:
init_context_embs = bi_encoder.encode(
    test_contexts,
    convert_to_tensor=True,
    show_progress_bar=True,
)

init_question_embs = bi_encoder.encode(
    test_questions,
    convert_to_tensor=True,
    show_progress_bar=True,
)

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
init_cos_scores = util.semantic_search(init_question_embs, init_context_embs, top_k=100)

In [ ]:
r5s = []
mrrs = []
rmaps = []
ndcgs = []

for i in range(len(test_questions)):
    pred = build_pred_mask_for_i(init_cos_scores, i=i, n=len(test_contexts))
    target = build_target_mask_for_i(i=i, n=len(test_contexts))

    pred = torch.tensor(pred)
    target = torch.tensor(target)

    r5 = retrieval_recall(preds=pred, target=target, top_k=5).item()
    mrr = retrieval_reciprocal_rank(preds=pred, target=target, top_k=5).item()
    rmap = retrieval_average_precision(preds=pred, target=target, top_k=5).item()
    ndcg = retrieval_normalized_dcg(preds=pred, target=target, top_k=10).item()

    r5s.append(r5)
    mrrs.append(mrr)
    rmaps.append(rmap)
    ndcgs.append(ndcg)

print(f'mean recall5@{len(docs)}: ', mean(r5s))
print('mean mrr: ', mean(mrrs))
print('mean rmAP: ', mean(rmaps))
print(f'mean ndcg10@{len(docs)}: ', mean(ndcgs))

mean recall5@891:  0.886
mean mrr:  0.7941333337575197
mean rmAP:  0.7941333337575197
mean ndcg10@891:  0.8264232175350189


### После дообучения

In [ ]:
r5s = []
mrrs = []
rmaps = []
ndcgs = []

for i in range(len(test_questions)):
    pred = build_pred_mask_for_i(cos_scores, i=i, n=len(test_contexts))
    target = build_target_mask_for_i(i=i, n=len(test_contexts))

    pred = torch.tensor(pred)
    target = torch.tensor(target)

    r5 = retrieval_recall(preds=pred, target=target, top_k=5).item()
    mrr = retrieval_reciprocal_rank(preds=pred, target=target, top_k=5).item()
    rmap = retrieval_average_precision(preds=pred, target=target, top_k=5).item()
    ndcg = retrieval_normalized_dcg(preds=pred, target=target, top_k=10).item()

    r5s.append(r5)
    mrrs.append(mrr)
    rmaps.append(rmap)
    ndcgs.append(ndcg)

print(f'mean recall5@{len(docs)}: ', mean(r5s))
print('mean mrr: ', mean(mrrs))
print('mean rmAP: ', mean(rmaps))
print(f'mean ndcg10@{len(docs)}: ', mean(ndcgs))

mean recall5@891:  0.886
mean mrr:  0.7941333337575197
mean rmAP:  0.7941333337575197
mean ndcg10@891:  0.8264232175350189


# Выводы

**Метрики близости**

|Метрика|Initial|Fine-Tuned|Изменение|
|-------|-------|----------|---------|
|Pearson (cosine)|0.874|0.923|+0.049|
|Spearman (cosine)|0.855|0.854|-0.001|

1. Метрики в целом высокие `0.85 - 0.9`
2. Метрики высокие и "из коробки": `P=0.87` и `S=0.85`
3. `Pearson` вырос на `5%`
* Модель почти идеально предсказывает сходство вопросов и контекстов по абсолютным значениям.
4. `Spearman` можно сказать, что `не изменился`
* Модель изначально хорошо ранжирует пары по релевантности
5. **Общий вывод по метрикам близости:**<br>Модель ранжирует примерно так же, но стала увереннее в своих оценках (что и видно по Pearson)
* Модель научилась лучше приближать вероятности, не ухудшив сортировку.
* **Fine-tuning не изменил порядок предсказаний**, но сделал значения cosine similarity более точными.

**Метрики ранжирования**

|Метрика|Initial|Fine-Tuned|Изменение|
|-------|-------|----------|---------|
|Recall5@1000|0.886|0.886|0|
|MRR|0.794|0.794|0|
|rmAP|0.794|0.794|0|
|NDCG10@1000|0.826|0.826|0|

1. Метрики никак не изменились<br>
Причины:
- Возможно дообучение было недостаточным
    - т.к. я взял не полный датасет, а лишь его часть
    - даже при этом модель обучалась 3 эпохи на протяжении получаса (colab ограничен😞)
    - возможно эпох было недостаточно
- Модель, несмотря на свой размер, уже из коробки отлично ранжирует ответы
    - `Recall@5 = 0.886` — уже высокая метрика.
2. В 5/5 различных вопросов, **все ответы в top-5 были верны**, отвечали либо полностью, либо относились к теме. Лишнего не было вообще.